# Parser 12.10 - Numeric Ternary Conditional

The goal of this version of the parser is to get it to understand the [ternary conditional](https://en.wikipedia.org/wiki/%3F:), the most common operator with three operands.

This operator evaluates one of two expressions, the value of which becomes its result. Which of the two gets evaluated depends on whether or not a third, conditional, expression is *True* or *False* (in programming languages with Boolean types) or simply zero or non-zero (in languages without Boolean types, such as [C](https://en.wikipedia.org/wiki/C_(programming_language)) \). In many languages it is written:

```Python
cond ? expr1 : expr2
```

If *cond* (also an expression) is non-zero, *expr1* is evaluated and its value is the result. If instead *cond* is zero, *expr2* is evaluated and its value is the result. Either way, the expression not evaluated is skipped over and has no effect.

>In [Python](https://en.wikipedia.org/wiki/Python_(programming_language)), a language with Boolean types, the ternary conditional is written more readably as:
```Python
expr1 if cond else expr2
```
Which expression gets evaluated and becomes the final result is based on *cond*, just as with the version above. Here it appears as the second of the three expressions rather than the first. It's also interesting to note here the use of **if** and **else** as operators instead of as the statements they are in every other context. A new type of polymorphism.

As with parentheses and braces, it is an error if one of the two characters that make up a single ternary conditional operator is not present, or if they both appear but in the wrong order.

Two or more ternary conditionals appearing consecutively are right associative and are parsed as:

cond1 ? expr1 : ( cond2 ? expr2 : expr3 )

>This form is also referred to as *nested* conditionals. The Python equivalent is:
```Python
expr1 if cond1 else expr2 if cond2 else expr3
```
which is in many ways easier to understand.

It is somewhat harder to answer the question of whether or not ternary conditionals can overlap. That is, if this is a legal expression:

```Python
cond1 ? cond2 ? expr1 : expr2 : expr3
```

>This form can't even be written in Python. A point in its favor.

## Libraries

In [31]:
import glob       # for searching directories
import math
import random     # for 'random()'

import re         # for regular expressions

## User output

In [32]:
visSep = '-------------'             # visual separator

def UIwriteln(this):
    '''write a single line to output'''
    print( f'{this}\n' )
    
def UIwriteSep():
    '''write a visual separator'''
    UIwriteln( visSep )

def UIshow(tag, value):
    '''write a tagged value to output'''
    UIwriteln( f'{tag}: {value}' )

def UIerror(this):
    '''write an error message to output'''
    UIshow( 'Error', this )

# Tracing

In [33]:
# flags: show trace of processing

showInteract = True          # default for interactive use
showBatch = False            # default for batch use

showTrace = None             # control flag

# Trace Output

def TOshow(mesg, text):
    '''write trace message to output if enabled'''
    if showTrace:
        UIshow( f'{mesg:15s}', text )
        
def TOstring(tag, this):
    
    if showTrace:
        TOshow( tag, ' '.join([str(e) for e in this]) )

# -----------------------
# Parse Tracing
# -----------------------

def PTshowexpr(this):

    TOshow( 'Parse', visSep )
    TOshow( 'Current Expr', this )

def PTshowparse(ok, res, opStk, typStk):

    if ok:
        TOstring( 'Current RPN', res )
        TOstring( 'Operator Stack', opStk )
        TOstring( 'Type Stack', typStk )

def PTshowtoken(this):

    if not this[0] == ' ':
        TOshow( "Found Token", this )

# -----------------------
# Evaluation Tracing
# -----------------------

def ETshowtoken(this):
    
    TOshow( 'Eval', visSep )
    TOshow( 'Current token', this )

def ETshoweval(stk):
    
    TOstring( 'Operand Stack', stk )


# Functions

In [34]:
def functionErr(nam, msg, val):
    UIerror( f'{nam}(): {msg} value: {val}')
    return ( False, None )

def fncAbs(val):
    '''absolute value of val'''
    return (True, abs(val))

def fncMax(args):
    '''max of two vals'''
    return (True, max(args))

def fncMin(args):
    '''min of two vals'''
    return (True, min(args))

def fncRand():
    '''random decimal'''
    return(True, random.random())

def fncRnd(val):
    '''rounded val'''
    if type(val) is not list:
        return (True, round(val))
    elif type(val[0]) is int:
        return (True, round(val[1], val[0]))
    else:
        return functionErr('ROUND', 'non-integer', val[0])

def fncSgn(val):
    '''sign of val'''
    return (True, 1 if val > 0 else -1 if val < 0 else 0 )

def fncSqt(val):
    '''square root of val'''
    if val >= 0:
        return (True, math.sqrt(val) )
    else:
        return functionErr('SQR', 'negative', val)
    
# known functions

fncDispatch = {
     'ABS': (fncAbs,  1, 1),
     'MAX': (fncMax,  2, None),
     'MIN': (fncMin,  2, None),
  'RANDOM': (fncRand, 0, 0 ),
   'ROUND': (fncRnd,  1, 2),
    'SIGN': (fncSgn,  1, 1),
    'SQRT': (fncSqt,  1, 1)
}


# Parser

In [35]:
# operands accepted:
# - decimal and hexadecimal floating point literals
# - scalar and array numeric variables
# - numeric functions with zero or more arguments

# operators accepted:
# - unary negation, plus
# - binary addition, subtraction, multiplication, division
# - grouping parentheses
# - logical not, equality, inequality
# - assignment and shortcut assignment
# - prefix and postfix increment and decrement
# - logical short circuit
# - ternary conditional

# errors detected:
# - unrecognized input
# - out of range numeric input
# - malformed expression

# result tuple:
# - (True, [parse])
# - (False, None)

class Parser(object):
    
    VERSIONNUMBER = '12.10'
    
    _FLTMAX =  4294967295                                  # 2**32-1
    _FLTMIN = -4294967296                                  # -(2**32)

    _expMax = {
        'P' : math.log2(_FLTMAX),                          # max base 2 exponent
        'E' : math.log10(_FLTMAX)                          # max base 10 exponent
    }

    _typeChkLst = {                                         # index is left-to-right, check right-to-left
        '?err': [  None,  'tererr'],
        'an2_': [  None,   'number', 'argsep' ],
         'f2n': ['number', 'fncsym'],
         'n2_': [  None,   'number'],
         'n2n': ['number', 'number'],
         'v2n': ['number', 'numsym'],
        'fn2n': ['number', 'fncsym', 'number'],
        'n?2n': ['number', 'C?-op',  'number'],
        'nn2n': ['number', 'number', 'number'],
        'vn2n': ['number', 'numsym', 'number'],
        'vn2v': ['numsym', 'numsym', 'number']
    }
    
    def __init__(self):
        pass
        
    def doparse(self, this):

        def parseErr(mesg, pos):
            '''report parse error'''
            UIerror(mesg)
            UIwriteln(f'>>> {this}')
            if pos > 0:
                UIwriteln(f'{"^^near here".rjust(pos)}')
            return False

        # initialize

        expr = this                # save to new variable but retain original for error reports
        start = 15                  # tracked so we can report where in an expression an error occurred
        token = None               # anything successfully matched
        ok = wantoperand = True    # flags
        result = []                # rpn expression
        opStk = [ ('EOE', 1) ]     # operator stack
        typStk = []                # type stack
        argStk = []                # function argument count stack
        

        def typeCheck(op, chk):
            '''type check operands'''
            nonlocal argStk
            check = list(self._typeChkLst[chk])       # list() to avoid aliasing
            while len(check) > 1:
                want = check.pop()
                
                # are we checking an argument separator ?
                
                if want == 'argsep':
                    if len(opStk) < 2 or opStk[-2][0] != 'B(':     # within a function call ?
                        return parseErr('Unexpected comma', 0)
                    else:
                        argStk[-1] += 1                            # one more argument
                        result.pop()                               # remove 'F,' from RPN
                        continue
                
                have, rpnPos, errPos = typStk.pop()
                
                # no problem ?
                
                if want == have:
                    continue
                
                # did we want a number ?
                
                if want == 'number':
                    
                    # convert symbols to numbers
                    
                    if have == 'numsym':
                        result.insert( rpnPos, 'U*' )
                                           
                # did we want a variable ?
                
                elif want == 'numsym':
                    return parseErr('Numeric variable expected', errPos)
                    
                # did we want a function name ?
                
                elif want == 'fncsym':
                    
                    if have != 'numsym':
                        return parseErr('Function name expected', errPos)
                    fnc = result[rpnPos-1]
                    if not fnc in fncDispatch:
                        return parseErr(f'Unknown function name: {fnc}', errPos)
                    _, mina, maxa = fncDispatch[fnc]
                    cnt = argStk.pop()
                    if (cnt < mina) or (maxa != None and cnt > maxa):
                        return parseErr(f'Bad argument count: {fnc}', errPos)
                    elif cnt > 1:
                        result.pop()                # remove 'B('
                        result.append(cnt)          # argument count
                        result.append('F()')        # multiple argument function call operator
                        
                # did we want a right hand '?' ?
                
                elif want == 'C?-op':
                    if opStk.pop()[0] != 'C?-':
                        return parseErr('":" without "?"', errPos)
                        
                # '?' without ':' ?
                
                elif want == 'tererr':
                    return parseErr('"?" without ":"', errPos)
                    
                # generic everything else
                
                else:
                    return parseErr('Type mismatch', errPos )
                        
            # push result type, RPN operator position, original operator position
                    
            restyp = check.pop()
            if restyp != None:
                typStk.append( (restyp, len(result), errPos) )
            return True
 
        def addOperand(op, typ):
            '''add operand to RPN'''
            result.append( op )
            typStk.append( (typ, len(result), start) )
            
        def addOperator():
            '''add operator to RPN'''            
            op, _, chk = opStk.pop()
            result.append( op )
            return typeCheck( op, chk )
            
        def popGEop(prec):
            '''pop operators of equal or greater precedence'''
            ok = True
            while ok and prec <= opStk[-1][1]:
                ok = addOperator()
            return ok
 
        def pushLeft(op, prec, chk):
            '''push left associative operator on stack'''
            if not popGEop(prec):
                return False
            opStk.append( (op, prec, chk) )
            return True

        def popGop(prec):
            '''pop operators of greater precedence'''
            ok = True
            while ok and prec < opStk[-1][1]:
                ok = addOperator()
            return ok
 
        def pushRight(op, prec, chk):
            '''push right associative operator on stack'''
            if not popGop(prec):
                return False
            opStk.append( (op, prec, chk) )
            return True

        def popUntil(op, prec):
            '''clear and check operator stack'''
            if not popGEop(prec):                       # type check failure ?
                return False

            topop = opStk.pop()[0]                      # found match ?
            if op == topop:
                return True
            
            elif op == '(':
                err = 'right parenthesis'
            elif op == '[':
                err = 'right bracket'
            elif topop == '(':
                err = 'left parenthesis'
            elif topop == '[':
                err = 'left bracket'
            else:
                err = 'EOE'
            
            return parseErr( f'Unmatched {err}', start )
   
        # operator dictionaries initialization
        
        _postOps = '[+]{2}|[-]{2}|\(\)'
        
        _postOp = {
            '++': (90, pushLeft, 'v2n'),
            '--': (90, pushLeft, 'v2n'),
            '()': (90, pushLeft, 'f2n')
        }

        _unOps = '!|[+]{1,2}|[-]{1,2}'

        _unOp =  {
            '-':  (80, pushRight, 'n2n'),
            '+':  (80, pushRight, 'n2n'),
            '!':  (80, pushRight, 'n2n'),
            '++': (80, pushRight, 'v2n'),
            '--': (80, pushRight, 'v2n'),
        }

        _binOps = '[-+*/]=?|[=!]?=|\[|\('

        _binOp = {
            '[':  (90, pushLeft,  'vn2v'),
            '(':  (90, pushLeft,  'fn2n'),
            '*':  (70, pushLeft,  'nn2n'),
            '/':  (70, pushLeft,  'nn2n'),
            '+':  (60, pushLeft,  'nn2n'),
            '-':  (60, pushLeft,  'nn2n'),
            '==': (50, pushLeft,  'nn2n'),
            '!=': (50, pushLeft,  'nn2n'),
            '=':  (10, pushRight, 'vn2n'),
            '*=': (10, pushRight, 'vn2n'),
            '/=': (10, pushRight, 'vn2n'),
            '+=': (10, pushRight, 'vn2n'),
            '-=': (10, pushRight, 'vn2n')
        }
        
        _condOps = '&&|[|]{2}|[?:]'
        
        _condOp = {
            '&&': [(40, pushLeft, 'n2_'),  (40, pushLeft, 'n2n')],
            '||': [(30, pushLeft, 'n2_'),  (30, pushLeft, 'n2n')],
            '?':  [(28, pushRight, 'n2_'), (24, pushRight, '?err')],
            ':':  [(26, pushRight, 'n2n'), (24, pushRight, 'n?2n')]
        }
      
        def convertFloat(fplit, base, capgrp):
            '''convert floating point literal to internal form'''
            
            def rangeErr():
                return parseErr(f'\'{fplit}\' is out of range', start)
            
            # collect the features of interest
                   
            p = re.search(capgrp, fplit.upper())
                            
            lint, lfrc, expbas, expsgn, lexp = p.group(1,2,4,5,6)
            
            # convert integer portion (if any)
 
            uint = 0
            if lint:
                p = re.search('[1-9A-F][0-9A-F]*', lint )
                if p != None:
                    for ch in p.group():
                        digval = '0123456789ABCDEF'.find(ch)
                        if uint <= (self._FLTMAX - digval)/base:
                            uint =  uint * base + digval
                        else:
                            return rangeErr()
                    
            # convert fractional portion (if any)
                    
            ufrc = 0
            if lfrc:
                fbase = 1
                for ch in lfrc:
                    digval = '0123456789ABCDEF'.find(ch)
                    fbase *= base
                    ufrc += digval/fbase
        
            if uint == self._FLTMAX and ufrc != 0:
                return rangeErr()
            
            # value so far
            
            uflt = uint + ufrc
 
            # convert exponent portion (if any)
            
            uexp = 0
            if lexp:
                for ch in lexp:
                    digval = '0123456789'.find(ch)
                    if uexp <= (self._expMax[expbas] - digval)/10:
                        uexp =  uexp * 10 + digval
                    else:
                        return rangeErr()
                    
            # adjust value by exponent (if any)
             
            if uexp:
                power = (2 if expbas == 'P' else 10) ** uexp
                if expsgn == '-':
                    uflt /= power
                elif uflt <= self._FLTMAX/power:
                    uflt *= power
                else:
                    return rangeErr()
                    
            addOperand( uflt, 'numlit' )
            return True

        def startsWith(regex):
            '''test if expression starts with given regular expression'''
            nonlocal expr, start, token

            p = re.match(regex, expr)
            if p == None:
                return False
            else:
                token = p.group()              # what we matched
                start += len(token)            # update to next match position in original string
                expr = expr[len(token):]       # "chop off" what we matched
                PTshowtoken(token)             # trace
                return True

        # top level main loop
              
        while ok and len(expr):

            _ = startsWith('[ ]+')                             # skip leading whitespace

            PTshowexpr(expr)                                   # trace

            # look for operand

            if wantoperand:

                if startsWith('[(]'):
                    '''left parenthesis ?'''
                    opStk.append( ('(', 2) )

                elif startsWith(_unOps):
                    '''right unary?'''
                    prec, assoc, check = _unOp[token]
                    assoc( 'U' + token, prec, check )

                else:

                    wantoperand = False                         # flip
                    
                    if startsWith('0[xX]([0-9a-fA-F]+([.][0-9a-fA-F]*)?|[.][0-9a-fA-F]+)([pP][-+]?[0-9]+)?'):
                        '''unsigned hexadecimal literal ?'''
                        ok = convertFloat(token, 16, '0X([0-9A-F]*)[.]?([0-9A-F]*)(([P])([-+])?([0-9]+))?')

                    elif startsWith('([0-9]+([.][0-9]*)?|[.][0-9]+)([eE][-+]?[0-9]+)?'):
                        '''unsigned decimal literal?'''
                        ok = convertFloat(token, 10, '([0-9]*)[.]?([0-9]*)(([E])([-+])?([0-9]+))?')
                        
                    elif startsWith('[a-zA-Z][_a-zA-Z0-9]*'):
                        '''numeric scalar variable or function name?'''
                        addOperand( token.upper(), 'numsym' )
                        
                    else:
                        '''malformed'''
                        ok = parseErr('Expecting operand', start)

            # look for operator

            else:

                if startsWith('[)\]]'):
                    '''expression terminator?'''
                    ok = popUntil( '(' if token == ')' else '[', 4 )
                        
                    
                elif startsWith(_postOps):
                    '''postfix operator?'''
                    prec, assoc, check = _postOp[token]
                    ok = assoc( 'P' + token, prec, check )
                    if ok and token == '()':
                        argStk.append( 0 )
 
                else:
                    
                    wantoperand = True                          # flip

                    if startsWith(_binOps):
                        '''binary operator?'''
                        prec, assoc, check = _binOp[token]
                        ok = assoc( 'B' + token, prec, check )
                        if ok and (token == '[' or token == '('):
                            opStk.append( (token, 2) )
                            if token == '(':
                                argStk.append( 1 )
                                
                    elif startsWith(_condOps):
                        '''conditional operator?'''
                        op = 'C' + token
                        precl, assocl, checkl = _condOp[token][0]
                        precr, assocr, checkr = _condOp[token][1]
                        ok = assocl( op + '+', precl, checkl) and assocr( op + '-', precr, checkr)

                    elif startsWith(','):
                        '''function argument separator?'''
                        ok = pushLeft( 'F,', 6, 'an2_' )

                    else:
                        '''malformed'''
                        ok = parseErr('Expecting operator', start)

            PTshowparse(ok, result, opStk, typStk )               # trace
 
        if ok and wantoperand:
            ok = parseErr('Unexpected end of expression', start ) # must be in 'wantoperator' state   

        if ok:
            ok = popUntil( 'EOE', 3 )                             # clear operator stack
            
        if ok:
            ok = typeCheck('LoneSymbol', 'n2n')                   # make sure final result is a number

        return (ok, result if ok else None)                       # done
       

### How it works

Perhaps the easiest way to approach parsing the ternary conditional is to work backwards. What does the final result of parsing have to look like in order for evaluation to proceed correctly?

Some thought suggests we have to place operators at three locations in the Reverse Polish: after the condition expression, after the *True* branch, and after the *False* branch:

```Python
cond C?+ expr1 C:+ expr2 C:-
```

*C?+* does nothing if *cond* is non-zero, otherwise it sets the evaluator to skip past *C:+* to the start of *expr2*.

*C:+* is only ever executed when *cond* is non-zero. If *C?+* allows evaluation of *expr1* to occur, when *C:+* is reached it unconditionally sets the evaluator to skip to just past *C:-*. It doesn't care about the value of *expr1* at all.

*C:-* never does anything. Its only purpose is to be a marker indicating where skipping should stop if *cond* was non-zero and *expr1* was evaluated. If instead *cond* was zero, *expr1* skipped and *expr2* evaluated, as with *C:+* it has no interest in the value of the result. When executed it simply acts as a do nothing operator, like *U+*.

>Technically both *C:+* and *C:-* could be considered *nonary*, operators with no operands. If that makes sense.

That seems to work. Now that we know what we want, how do we arrange for it?

One way is to split both **?** and **:** into two operators each, one left and one right. Then we can simply add them to *_condOps* and *_condOp{}* along with **&&** and **||**. We don't have change anything in the main parse loop at all.

However there are a few twists. The first has to do with precedences. All the new operators have precedences that are lower than logical OR but higher than any assignment operator. This keeps the ternary conditional in its correct overall precedence. But unlike **&&** and **||**, they are all right associative, the left and right hand versions have different precedences, and both right hand versions have the same precedence.

Another novelty is the introduction of two new type checks, *'n?2n'* and *'?err'*, along with modifications to *typeCheck()* to implement them.

The *'n?2n'* check is associated with the *C:-* operator. Since splitting **?** and **:** into two unary operators each gives us four new operators when we only want three. We deal with that using this check. It works like this:

Since the left and right hand versions of either operator are pushed one right after the other, the lower precedence of each right hand version will immediately pop its partner left hand version into the Reverse Polish. Assuming a syntactically correct expression, after finding **?**, the Reverse Polish will look like:

```Python
cond C?+
```

and the operator stack will look like:

```Python
C?-
```

When the matching **:** is found, these change to:

```Python
cond C?+ expr1 C:+
```

and

```Python
C?- C:-
```

The right associative *C?-* and *C:-* have the same precedence, so *C:-* does not remove *C?-* from the operator stack. But we don't actually want *C?-* in the Reverse Polish. When *C:-* is eventually popped off the operator stack and gets type checked, the Reverse Polish looks like:

```Python
cond C?+ expr1 C:+ expr2 C:-
```

and the operator stack like:

```Python
C?-
```

During the type check, we want to find that *expr2* is numeric. But we also want to find a *C?-* at the top of the operator stack. If there is, we take this opportunity to throw it away. However if we don't find one, we never came across the **?** that should have put it there. So we have **:** without **?**, which is an error.

What if we find a **?** but no matching **:**? Then when *C?-* is eventually popped from the operator stack, its *'?err'* type check unconditionally reports that error.

So *C?-*, while never appearing in the final Reverse Polish, nevertheless serves a vital error detection role during parsing.

#### Nested conditionals

```Python
cond1 ? expr1 : cond2 ? expr2 : expr3
```

is parsed into:

```Python
cond1 C?+ expr1 C:+ cond2 C?+ expr2 C:+ expr3 C:- C:-
```

which evaluates properly whatever combination of *cond1* and *cond2* we test. The parser has no difficulty doing this.

For expressions like:

```Python
cond1 ? cond2 ? expr1 : expr2 : expr3
```

to evaluate properly, they must be parsed into:

```
cond1 C?+ cond2 C?+ expr1 C:+ expr2 C:- C:+ expr3 C:-
```

which evaluates correctly whatever combination of *cond1* and *cond2* we test. However if the original expression is written in this form, this version of the parser cannot produce this RPN.

>What happens is a *C:-* gets placed on top of a *C:-* on the operator stack. An error is reported when the top *C:-* is typed checked and finds *C:-* instead of *C?-*.

To produce the required RPN, parentheses must be explicitly employed:

```Python
cond1 ? ( cond2 ? expr1 : expr2 ) : expr3
```

>Does this even matter? Unlike consecutive conditionals, overlapped conditionals never appear in Kernighan and Ritchie’s [The C Programming Language](https://www.goodreads.com/book/show/515601.The_C_Programming_Language), nor in Harbison and Steele’s [C: A Reference Manual](https://www.goodreads.com/book/show/946107.C). In practice they are very rarely used.

>However it is also true that even without parentheses compilers such as [Turbo C 2.0](https://en.wikipedia.org/wiki/Borland_Turbo_C) readily parse it as:
```Python
cond1 ? ( cond2 ? expr1 : expr2 ) : expr3
```
>We *can* actually do the same by treating both **?** and **:** individually as unique operators rather than as akin to **&&** and **||**. As with this version of the parser, **?** and **:** are split into four left and right operators of different precedences. However that isn't enough by itself to ensure that the four are pushed on and popped off the operator stack at the proper times. The parser must perform additional special case actions (hence the need for individual treatment). The increased complexity is inelegant and not easy to understand. Other than the additional ability to correctly parse a very limited use case where explicit parentheses are not used, the effort has little to recommend it.

>For the truly masochistic, the gory details of how to do it can be found [here](https://anexpressionparser.wordpress.com/2015/11/06/parser-8-00-numeric-ternary-conditional/)


# Evaluator

In [36]:
# operators handled:
# - unary negation, plus
# - binary addition, subtraction, multiplication, division
# - logical negation, equality and inequality
# - variable name de-reference
# - variable assignment and shortcut assignment
# - prefix and postfix increment and decrement
# - numeric array assignment and de-reference
# - function calls with one or more arguments

# errors detected:
# - out of range
# - division by zero
# - invalid function arguments

# return tuple:
# - (True, result)
# - (False, None)

class Evaluator(Parser):
    
    def __init__(self):
        self._symTable = dict()
 
    def doeval(self, rpn):
 
        def inRange(ok, val):
            '''range check test result'''
            if ok:
                stk.append( val )
            else:
                UIerror( 'Evaluation result out of range' )
            return ok
        
        def pushOperand(val):
            '''push operand on stack'''
            stk.append( val )
            return True
        
        def setSkip(down, up):
            '''set skip flags'''
            nonlocal skipLevel, downToken, upToken
            skipLevel = 1
            downToken = down
            upToken = up
            
        def checkSkip(skip, val, down, up):
            if skip:
                setSkip(down, up)
                stk.append(val)
            return True
                
        def logLftAnd(val):
            '''left branch of logical AND'''
            return checkSkip(val == 0, 0, 'C&&-', 'C&&+')
        
        def logLftOr(val):
            '''left branch of logical OR'''
            return checkSkip(val != 0, 1, 'C||-', 'C||+')
                
        def logRgt(val):
            '''right branch of logical AND and OR'''
            return pushOperand(1 if val != 0 else 0)

        def terCond(val):
            '''ternary condition'''
            if not val:
                setSkip('C:+', 'C?+')
            return True
        
        def terTrue(val):
            '''end of ternary true branch'''
            return checkSkip(True, val, 'C:-', 'C:+')
        
#        def terFalse(val):
#           '''end of ternary false branch'''
#           return pushOperand(val)
        
        def binAryNdx(ndx, nam):
            '''create array index'''
            return inRange( 0 <= ndx <= self._FLTMAX, f'{nam}_{ndx}' )
                     
        def binVal(val, var):
            '''assign value to variable'''
            self._symTable[var] = val
            return pushOperand(val)
        
        def unVal(var):
            '''variable name value'''
            return pushOperand(self._symTable[var] if var in self._symTable else 0)

#        def unPlu(arg):
#           '''unary plus'''
#           return pushOperand( arg )

        def unNeg(arg):
            '''unary negation'''
            return inRange( arg != self._FLTMIN, -arg )
        
        def unNot(arg):
            '''logical not'''
            return pushOperand( not arg )

        def binAdd(rgt, lft):
            '''binary addition'''
            if lft >= 0:
                return inRange( rgt <= self._FLTMAX - lft, lft+rgt )       
            else:
                return inRange( rgt >= self._FLTMIN - lft, lft+rgt )
 
        def binSub(rgt, lft):
            '''binary subtraction'''
            if lft >= 0:
                return inRange( lft - self._FLTMAX <= rgt, lft-rgt )
            elif rgt >= 0:
                return inRange( lft - self._FLTMIN >= rgt, lft-rgt )
            else:
                return inRange( lft <= self._FLTMAX + rgt, lft-rgt )
 
        def binMul(rgt, lft):
            '''binary multiplication'''
            if abs(lft) <= 1 or abs(rgt) <= 1:
                return pushOperand( lft * rgt )

            elif lft > 0:
                if rgt > 0:
                    return inRange( rgt <= self._FLTMAX / lft, lft * rgt )
                else:
                    return inRange( rgt >= self._FLTMIN / lft, lft * rgt )

            elif rgt > 0:
                return inRange( rgt <= self._FLTMIN / lft, lft * rgt )
  
            else:
                return inRange( rgt >= self._FLTMAX / lft, lft * rgt )
 
        def binDiv(rgt, lft):
            '''binary division'''
            if abs(rgt) >= 1:
                return pushOperand( lft/rgt )
            
            elif rgt > 0:
                if lft > 0 :
                    return inRange( lft <= self._FLTMAX * rgt, lft / rgt )
                else:
                    return inRange( lft >= self._FLTMIN * rgt, lft / rgt )
            
            elif rgt < 0:
                if lft > 0:
                    return inRange( lft <= self._FLTMIN * rgt, lft / rgt )
                else:
                    return inRange( lft >= self._FLTMAX * rgt, lft / rgt )

            else:
                UIerror( 'Division by zero' )
                return False
            
        def binEqu(rgt, lft):
            '''logical equality'''
            return pushOperand( lft == rgt )
        
        def binNeq(rgt, lft):
            '''logical inequality'''
            return pushOperand( lft != rgt )
        
        def binShortVal(val, var, op):
            '''shortcut assignment'''
            _ = unVal(var)                           # put the value of the variable on the stack
            if op(val, stk.pop()):                   # perform the arithmetic
                return binVal(stk.pop(), var)        # if successful,  also assign result to variable
            return False
        
        def binAddVal(val, var):
            return binShortVal(val, var, binAdd)
        
        def binSubVal(val, var):
            return binShortVal(val, var, binSub)
        
        def binMulVal(val, var):
            return binShortVal(val, var, binMul)
        
        def binDivVal(val, var):
            return binShortVal(val, var, binDiv)
        
        def unPfxInc(var):
            return binShortVal(1, var, binAdd)
        
        def unPfxDec(var):
            return binShortVal(1, var, binSub)
        
        def unPostFix(val, var):
            '''postfix inc/dec'''
            _ = unVal(var)                             # push current value of variable on stack
            ok = binShortVal(val, var, binAdd)         # update value of variable
            stk.pop()                                  # remove updated value from stack (if error, removes first push)
            return ok
                
        def unPstInc(var):
            return unPostFix(1, var)
            
        def unPstDec(var):
            return unPostFix(-1, var)
        
        def binFnc(val, fnc):
            '''single argument function call'''
            ok, val = fncDispatch[fnc][0](val)
            return pushOperand(val) if ok else False
                
        def multiFnc(cnt):
            '''multiple argument function call'''
            args = []
            while cnt > 0:
                args.append( stk.pop() )
                cnt -= 1
            ok, val = fncDispatch[stk.pop()][0](args)               
            return pushOperand(val) if ok else False
        
        def zeroFnc(name):
            '''zero argument function call'''
            ok, val = fncDispatch[name][0]()
            return pushOperand(val) if ok else False
        
        # initialize
                                
        unDispatch = {
            'U*': unVal,
            'U-': unNeg, 'U+': pushOperand,
            'U!': unNot,
            'U++': unPfxInc,   'U--': unPfxDec,
            'P++': unPstInc,   'P--': unPstDec,
            'F()': multiFnc,   'P()': zeroFnc,
            'C&&+': logLftAnd, 'C&&-': logRgt,
            'C||+': logLftOr,  'C||-': logRgt,
            'C?+': terCond,
            'C:+': terTrue, 'C:-': pushOperand
        }
        
        binDispatch = {
            'B+': binAdd, 'B-': binSub,
            'B+=': binAddVal, 'B-=': binSubVal,
            'B*': binMul, 'B/': binDiv,
            'B*=': binMulVal, 'B/=': binDivVal,
            'B==': binEqu, 'B!=': binNeq,
            'B=': binVal,
            'B[': binAryNdx, 'B(': binFnc
        }
  
        skipLevel = 0
        downToken = upToken = None
        stk = []
        ok = True

        # main loop
        
        TOstring('Input', rpn)
        
        for v in rpn:

            ETshowtoken(v)

            if skipLevel > 0:
                TOshow('Skip level', f'{skipLevel}')
                if v == downToken:
                    skipLevel -= 1
                elif v == upToken:
                    skipLevel += 1
            
            elif v in binDispatch:
                ok = binDispatch[v](stk.pop(), stk.pop())
                
            elif v in unDispatch:
                ok = unDispatch[v](stk.pop())
                
            else:
                stk.append( v )

            if not ok:
                return ( False, None )

            ETshoweval( stk )

        return ( True, stk.pop() )


### How it works

We break *setSkip()* out of *checkSkip()* so we can use it for *C?+*, which does not push a value on the operand stack.

*C?+* sets the evaluator skipping to just after *C:+* if the value it is given is zero (*False*). Otherwise it allows evaluation of the *True* branch to proceed.

*C:+* can use *checkSkip()* because it has to unconditionally set the evaluator skipping to just after its matching *C:-* and also push the value it is given back on the operand stack.

*C:-* just needs to push the value it is given back on the operand stack, so we point it directly at *pushOperand()*.

>We could avoid both redundant pushes by introducing a dispatch dictionary just for operators that don't need to use the operand stack. The cost would be a fairly frequent check made each time through the main evaluation loop.

## Running the parser

In [37]:
passCnt = failCnt = 0                          # most useful for test input files, but never any harm

myParser = myEvaluator = None                  # where we keep instances of our classes

def startUp(flag):
    '''begin execution'''
    global passCnt, failCnt, showTrace
    global myParser, myEvaluator
    if not myParser:
        myParser = Parser()
    if not myEvaluator:
        myEvaluator = Evaluator()
    UIshow( 'Parser', myParser.VERSIONNUMBER )
    passCnt = failCnt = 0
    showTrace = flag
    
def shutDown():
    '''terminate execution'''
    UIwriteSep()
    UIshow( 'Pass', passCnt )
    UIshow( 'Fail', failCnt )
    
# run parser
        
def parseOne(this):
    '''parse/evaluate one expression'''
    global passCnt, failCnt
    UIwriteSep()
    neg = this[0] == '@'
    if neg:
        this = this[1:]
    UIshow( 'Input', this )
    ok, res = myParser.doparse( this )
    if ok:
        UIshow( 'Final Parse', res )
        ok, res = myEvaluator.doeval( res )
        if ok:
            UIshow( 'Final Eval', res )
        if neg:
            ok = not ok
    if ok:
        passCnt += 1
    else:
        failCnt += 1

## Interactive use

In [38]:
def parse():
    
    startUp(showInteract)
    while True:
        inp = input( 'Expression: ' )
        UIwriteln( '' )                      # looks better with a blank line here
        if inp.upper()[0] == 'Q':
            break
        elif inp.strip():
            parseOne( inp )
    shutDown()

## Batch processing

In [39]:
testDir = '..\\ParserTest\\'            # directory holding test input files (empty string if same as notebook directory)

# convert current version number to match test file numbers
# - done this way so we can update only the version number and everything still works

def currNum():
    
    head = myParser.VERSIONNUMBER[:len(myParser.VERSIONNUMBER)-3]
    tail = myParser.VERSIONNUMBER[-2:]
    return f'{head:0>2}{tail}'

# make full path name to test file

def makePath(typ, num):
    return f'{testDir}{typ}{num}.txt'

# run one test

def runTest(this):
    
    UIwriteln(f'Parser {myParser.VERSIONNUMBER} vs {this[-12:-4]}')
    
    myEvaluator._symTable.clear()
    
    with open(this) as f:
        data = f.readlines()
    for line in data:
        test = line.strip()
        if test and test[0] != '#':         # skip blank and comment lines
            parseOne(test)
    
# run a test of current or specified version which should succeed
    
def good(num='curr'):
  
    startUp(showBatch)
    runTest(makePath('pass', currNum() if num == 'curr' else num))
    shutDown()
    
# run a test of current or specified version which should fail

def bad(num='curr'):
    
    startUp(showBatch)
    runTest(makePath('fail', currNum() if num == 'curr' else num))
    shutDown()
    
# run regression test against current and all previous test files

def regress():
            
    UIwriteln('PASS tests')
    
    startUp(showBatch)                       # must create objects before we can access variables inside them 
    currFn = makePath('pass', currNum())
    failed = []
    fnlist = glob.glob(f'{testDir}pass????.txt')
    for fn in fnlist:
        if fn <= currFn:
            atstart = failCnt
            runTest(fn)
            if atstart < failCnt:
                failed.append(fn)               
    shutDown()
    
    UIwriteln('FAIL tests')
    
    startUp(showBatch)
    currFn = makePath('fail',currNum())
    passed = []
    fnlist = glob.glob(f'{testDir}fail????.txt')
    for fn in fnlist:
        if fn <= currFn:
            atstart = passCnt
            runTest(fn)
            if atstart < passCnt:
                passed.append(fn)                
    shutDown()
    
    if not len(failed):
        UIwriteln('All pass tests succeded')
    else:
        UIwriteln('Pass tests which failed')
        for fn in failed:
            UIwriteln(f'  {fn}')
            
    if not len(passed):
        UIwriteln('All fail tests succeded')
    else:
        UIwriteln('Fail tests which passed')
        for fn in passed:
            UIwriteln(f'   {fn}')
              

# Testing the parser

In [ ]:
parse()       # interactive, one expression at a time

In [ ]:
good()        # run current parser against its own pass test. Use good('1234') to run against specific pass test.

In [ ]:
bad()         # run current parser against its own fail test. Use bad('5678') to run against specific fail test.

In [ ]:
regress()     # run parser against all previous tests